# Introduction

The provided notebook is a Python program for cryptocurrency data analysis and portfolio optimization. It uses historical price data and market cap data for selected cryptocurrencies to perform portfolio optimization using the Efficient Frontier approach. The script aims to calculate the profit/loss of the portfolio over different time periods and with varying numbers of coins.

# Notebook
## Import libs

In [1]:
import sys
sys.path.append("../")

import itertools
from time import time
from copy import deepcopy
from datetime import datetime


import numpy as np
import pandas as pd


import app_wrapper

## Inputs

In [2]:
start = time()

top_100 = False

# you place 100e each month per category for the past n years
budget = 100

save_dir = './legacy_data'

market_phase = "Bull"

most_recent = True

In [3]:
# Define the  dates
latest_update = "27/07/2023"

# Halving Dates
halving_2k16 = "09/07/2016"
halving_2k20 = "11/05/2020"

# Bear Dates 
bear_2k18 = "01/01/2018"
bear_2k22 = "01/01/2022"



market = dict()

market['Bull'] = {
    "2nd": (app_wrapper.convert_date_to_number(latest_update, halving_2k16) + 365,
             app_wrapper.convert_date_to_number(latest_update, halving_2k16) - 365),
    "3rd": (app_wrapper.convert_date_to_number(latest_update, halving_2k20) + 365,
                           app_wrapper.convert_date_to_number(latest_update, halving_2k20) - 365)}
market["Bear"] = {
    "2nd" : (app_wrapper.convert_date_to_number(latest_update, bear_2k18), 
              app_wrapper.convert_date_to_number(latest_update, halving_2k20) + 365),
    "3rd" : (app_wrapper.convert_date_to_number(latest_update, bear_2k22), 0)}# Now

In [4]:
if most_recent:
    delta_buy = market[market_phase]["3rd"][0]
    delta_sell = market[market_phase]["3rd"][1]
else:
    delta_buy = market[market_phase]["2nd"][0]
    delta_sell = market[market_phase]["2nd"][1]      
    
    
print(f"\ndelta_buy : {delta_buy}")
print(f"\ndelta_sell : {delta_sell}")


delta_buy : 1537

delta_sell : 807


In [5]:
n_days_vector = np.arange(delta_buy, delta_sell, -30)
# df_mc = pd.read_csv('./legacy_data/All_cryptos_market_cap.csv', index_col=0)
df_mc = pd.read_csv(f'{save_dir}/All_cryptos_market_cap.csv', index_col=0)

## Optimization parameters

In [6]:
possible_mu_methods = ["mean", "exp", "capm"]
possible_cov_methods = ["sample", "exp"]
possible_obj_functions = ["quadratic", "sharpe", "min_volat"]
compounding_options = [True, False]

# Get all combinations of mu_method, cov_method, and obj_function
all_combinations = list(itertools.product(possible_mu_methods, possible_cov_methods, possible_obj_functions, compounding_options))

## Variables initialization

In [7]:
maxpl2 = 0
maxpl10 = 0
maxpl20 = 0


best_portf_2 = None
best_portf_10 = None
best_portf_20 = None

best_res_2 = None
best_res_10 = None 
best_res_20 = None  

optimized_variables = dict()

# Create a list of inputs dictionaries for different numbers of coins
inputs_list = [
    {
        "n_coins": 2,
        "mu_method": None,
        "cov_method": None,
        "obj_function": None,
        "compounding": None,
        "data": df_mc,
        "top_100": top_100,
        "budget": budget,
        "days_vector": n_days_vector,
        "sell_day": delta_sell,
        "save_dir": save_dir
    },
    {
        "n_coins": 10,
        "mu_method": None,
        "cov_method": None,
        "obj_function": None,
        "compounding": None,
        "data": df_mc,
        "top_100": top_100,
        "budget": budget,
        "days_vector": n_days_vector,
        "sell_day": delta_sell,
        "save_dir": save_dir
    },
    {
        "n_coins": 20,
        "mu_method": None,
        "cov_method": None,
        "obj_function": None,
        "compounding": None,
        "data": df_mc,
        "top_100": top_100,
        "budget": budget,
        "days_vector": n_days_vector,
        "sell_day": delta_sell,
        "save_dir": save_dir
    }
]

## Run Analysis

In [8]:
# Test the portfolio_optimization function with all combinations
start = time()

maxpl2, maxpl10, maxpl20 = float("-inf"), float("-inf"), float("-inf")
best_portf_2, best_portf_10, best_portf_20 = None, None, None
best_res2, best_res10, best_res20 = None, None, None

optimized_variables = {}

for mu_method, cov_method, obj_function, compounding in all_combinations:
    for inputs in inputs_list:
        inputs["mu_method"] = mu_method
        inputs["cov_method"] = cov_method
        inputs["obj_function"] = obj_function
        inputs["compounding"] = compounding

        n_coins = inputs["n_coins"]
        print(f'\n===== {n_coins} COINS ======\n')

        p_l, results, portf = app_wrapper.calculate_profit(inputs)

        if n_coins == 2 and p_l >= maxpl2:
            maxpl2 = p_l
            best_portf_2 = portf
            best_res2 = results

            optimized_variables[f"{n_coins}_coins"] = {
                "mu": mu_method,
                "s": cov_method,
                "obj_function": obj_function,
                "compounding": compounding
            }

        elif n_coins == 10 and p_l >= maxpl10:
            maxpl10 = p_l
            best_portf_10 = portf
            best_res10 = results

            optimized_variables[f"{n_coins}_coins"] = {
                "mu": mu_method,
                "s": cov_method,
                "obj_function": obj_function,
                "compounding": compounding
            }

        elif n_coins == 20 and p_l >= maxpl20:
            maxpl20 = p_l
            best_portf_20 = portf
            best_res20 = results

            optimized_variables[f"{n_coins}_coins"] = {
                "mu": mu_method,
                "s": cov_method,
                "obj_function": obj_function,
                "compounding": compounding
            }

end = time()
print(f'Time elapsed: {(end - start) / 60}')



===== 2 COINS ======

Expected annual return: 184.8%
Annual volatility: 116.9%
Sharpe Ratio: 1.56
Invest 100.0

 Profit Loss: 1856.317902341897e

Expected annual return: 207.4%
Annual volatility: 124.5%
Sharpe Ratio: 1.65
Invest 100.0

 Profit Loss: 1490.119687715958e

Expected annual return: 202.7%
Annual volatility: 119.4%
Sharpe Ratio: 1.68
Invest 100.0

 Profit Loss: 1289.088367624569e

Expected annual return: 166.7%
Annual volatility: 106.3%
Sharpe Ratio: 1.55
Invest 100.0

 Profit Loss: 1467.5693486994994e

Expected annual return: 150.2%
Annual volatility: 101.2%
Sharpe Ratio: 1.46
Invest 100.0

 Profit Loss: 1586.7588978050908e

Expected annual return: 150.6%
Annual volatility: 104.3%
Sharpe Ratio: 1.43
Invest 100.0

 Profit Loss: 1599.895340961234e

Expected annual return: 141.7%
Annual volatility: 100.7%
Sharpe Ratio: 1.39
Invest 100.0

 Profit Loss: 1609.2076776513181e

Expected annual return: 124.7%
Annual volatility: 95.7%
Sharpe Ratio: 1.28
Invest 100.0

 Profit Loss: 184

Expected annual return: 418.3%
Annual volatility: 118.0%
Sharpe Ratio: 3.53
Invest 100.0

 Profit Loss: 2298.319217646685e

Expected annual return: 467.2%
Annual volatility: 145.6%
Sharpe Ratio: 3.20
Invest 99.99999999999999

 Profit Loss: 2215.9225698180294e

Expected annual return: 433.0%
Annual volatility: 137.4%
Sharpe Ratio: 3.14
Invest 100.0

 Profit Loss: 2143.6020342847355e

Expected annual return: 421.0%
Annual volatility: 133.1%
Sharpe Ratio: 3.15
Invest 100.0

 Profit Loss: 1917.2628583689184e

Expected annual return: 446.0%
Annual volatility: 141.7%
Sharpe Ratio: 3.13
Invest 100.0

 Profit Loss: 1628.0724567067005e

Expected annual return: 435.7%
Annual volatility: 128.5%
Sharpe Ratio: 3.37
Invest 100.0

 Profit Loss: 1253.1689560263771e

Expected annual return: 751.9%
Annual volatility: 153.0%
Sharpe Ratio: 4.90
Invest 100.0

 Profit Loss: 180.69756974230714e

Expected annual return: 3453.0%
Annual volatility: 208.7%
Sharpe Ratio: 16.54
Invest 100.0

 Profit Loss: -44.7924

Expected annual return: 296.7%
Annual volatility: 117.9%
Sharpe Ratio: 2.50
Invest 99.99999999999999

 Profit Loss: 3165.3068299355928e

Expected annual return: 282.8%
Annual volatility: 114.3%
Sharpe Ratio: 2.46
Invest 100.0

 Profit Loss: 3270.023430985805e

Expected annual return: 298.3%
Annual volatility: 110.2%
Sharpe Ratio: 2.69
Invest 100.0

 Profit Loss: 1926.0856204231466e

Expected annual return: 286.3%
Annual volatility: 105.9%
Sharpe Ratio: 2.68
Invest 100.0

 Profit Loss: 1892.9301388574186e

Expected annual return: 267.8%
Annual volatility: 107.2%
Sharpe Ratio: 2.48
Invest 100.0

 Profit Loss: 2704.603542348777e

Expected annual return: 266.1%
Annual volatility: 103.4%
Sharpe Ratio: 2.55
Invest 99.99999999999999

 Profit Loss: 2216.391542325346e

Expected annual return: 260.2%
Annual volatility: 99.4%
Sharpe Ratio: 2.60
Invest 100.0

 Profit Loss: 1971.1230624814036e

Expected annual return: 251.6%
Annual volatility: 97.1%
Sharpe Ratio: 2.57
Invest 100.0

 Profit Loss: 21

Expected annual return: 898.4%
Annual volatility: 151.9%
Sharpe Ratio: 5.90
Invest 100.0

 Profit Loss: 8.555252067687839e


===== 20 COINS ======

Expected annual return: 1784.7%
Annual volatility: 184.1%
Sharpe Ratio: 9.68
Invest 100.0

 Profit Loss: 3117.569621286003e

Expected annual return: 1996.9%
Annual volatility: 181.7%
Sharpe Ratio: 10.98
Invest 100.01299675081229

 Profit Loss: 1983.2864340687952e

Expected annual return: 1444.7%
Annual volatility: 151.3%
Sharpe Ratio: 9.53
Invest 99.99999999999999

 Profit Loss: 2031.4837844811236e

Expected annual return: 1299.1%
Annual volatility: 152.2%
Sharpe Ratio: 8.52
Invest 100.0

 Profit Loss: 2126.0647545126008e

Expected annual return: 973.2%
Annual volatility: 146.7%
Sharpe Ratio: 6.62
Invest 99.99999999999999

 Profit Loss: 2545.0455850308963e

Expected annual return: 771.0%
Annual volatility: 139.9%
Sharpe Ratio: 5.50
Invest 99.99999999999999

 Profit Loss: 3357.5651149432924e

Expected annual return: 710.3%
Annual volatility:

Expected annual return: 237.5%
Annual volatility: 102.8%
Sharpe Ratio: 2.29
Invest 100.0

 Profit Loss: 1336.1579718869718e

Expected annual return: 240.5%
Annual volatility: 87.2%
Sharpe Ratio: 2.73
Invest 100.0

 Profit Loss: 1089.2393723772184e

Expected annual return: 233.3%
Annual volatility: 87.0%
Sharpe Ratio: 2.66
Invest 100.0

 Profit Loss: 984.4452482382401e

Expected annual return: 229.5%
Annual volatility: 89.9%
Sharpe Ratio: 2.53
Invest 100.0

 Profit Loss: 1075.7321745477268e

Expected annual return: 229.4%
Annual volatility: 89.9%
Sharpe Ratio: 2.53
Invest 100.0

 Profit Loss: 919.491796652799e

Expected annual return: 225.7%
Annual volatility: 88.9%
Sharpe Ratio: 2.52
Invest 100.0

 Profit Loss: 787.9733255972038e

Expected annual return: 234.6%
Annual volatility: 89.3%
Sharpe Ratio: 2.60
Invest 100.0

 Profit Loss: 561.3266572246414e

Expected annual return: 259.6%
Annual volatility: 92.5%
Sharpe Ratio: 2.78
Invest 100.00000000000001

 Profit Loss: 139.86074589773995e


...

Expected annual return: 250.0%
Annual volatility: 73.6%
Sharpe Ratio: 3.37
Invest 100.00000000000001

 Profit Loss: 765.6448295962225e

Expected annual return: 246.0%
Annual volatility: 67.6%
Sharpe Ratio: 3.61
Invest 99.99999999999999

 Profit Loss: 962.6270478631097e

Expected annual return: 234.6%
Annual volatility: 72.0%
Sharpe Ratio: 3.23
Invest 100.0

 Profit Loss: 832.5234127251638e

Expected annual return: 252.7%
Annual volatility: 119.6%
Sharpe Ratio: 2.10
Invest 100.0

 Profit Loss: 458.8077880406286e

Expected annual return: 246.7%
Annual volatility: 105.5%
Sharpe Ratio: 2.32
Invest 100.0

 Profit Loss: 467.39896016470084e

Expected annual return: 236.4%
Annual volatility: 91.4%
Sharpe Ratio: 2.57
Invest 100.0

 Profit Loss: 488.14017811504743e

Expected annual return: 202.6%
Annual volatility: 71.4%
Sharpe Ratio: 2.81
Invest 100.0

 Profit Loss: 734.7687904524156e

Expected annual return: 212.9%
Annual volatility: 71.1%
Sharpe Ratio: 2.97
Invest 100.0

 Profit Loss: 1113.29

Expected annual return: 157.6%
Annual volatility: 86.8%
Sharpe Ratio: 1.79
Invest 100.0

 Profit Loss: 31.919999106276688e


===== 10 COINS ======

Expected annual return: 216.2%
Annual volatility: 77.8%
Sharpe Ratio: 2.75
Invest 100.00000000000001

 Profit Loss: 1143.443617428835e

Expected annual return: 220.2%
Annual volatility: 84.8%
Sharpe Ratio: 2.57
Invest 100.0

 Profit Loss: 809.0222329779604e

Expected annual return: 255.0%
Annual volatility: 86.2%
Sharpe Ratio: 2.94
Invest 100.0

 Profit Loss: 898.3495114994969e

Expected annual return: 210.3%
Annual volatility: 81.6%
Sharpe Ratio: 2.55
Invest 100.0

 Profit Loss: 1070.198119257999e

Expected annual return: 202.9%
Annual volatility: 76.4%
Sharpe Ratio: 2.63
Invest 100.0

 Profit Loss: 1277.4943986346254e

Expected annual return: 190.4%
Annual volatility: 76.4%
Sharpe Ratio: 2.47
Invest 100.0

 Profit Loss: 1072.2608275915727e

Expected annual return: 188.4%
Annual volatility: 72.5%
Sharpe Ratio: 2.57
Invest 100.0000000000000

Expected annual return: 92.0%
Annual volatility: 74.6%
Sharpe Ratio: 1.21
Invest 100.0

 Profit Loss: 524.9165756814983e

Expected annual return: 99.5%
Annual volatility: 68.9%
Sharpe Ratio: 1.41
Invest 100.0

 Profit Loss: 406.02458433206687e

Expected annual return: 96.7%
Annual volatility: 65.5%
Sharpe Ratio: 1.44
Invest 100.0

 Profit Loss: 453.4693117866045e

Expected annual return: 94.2%
Annual volatility: 58.7%
Sharpe Ratio: 1.57
Invest 100.0

 Profit Loss: 437.5182971169148e

Expected annual return: 96.8%
Annual volatility: 53.7%
Sharpe Ratio: 1.77
Invest 100.0

 Profit Loss: 318.46882021129466e

Expected annual return: 104.8%
Annual volatility: 59.8%
Sharpe Ratio: 1.72
Invest 100.0

 Profit Loss: 195.68077413401224e

Expected annual return: 116.0%
Annual volatility: 61.4%
Sharpe Ratio: 1.86
Invest 100.0

 Profit Loss: 122.55785942811788e

Expected annual return: 120.4%
Annual volatility: 76.7%
Sharpe Ratio: 1.54
Invest 100.0

 Profit Loss: 76.20959913198075e

Expected annual r

Expected annual return: 105.2%
Annual volatility: 71.2%
Sharpe Ratio: 1.45
Invest 100.0

 Profit Loss: 732.0456518808553e

Expected annual return: 116.7%
Annual volatility: 70.4%
Sharpe Ratio: 1.63
Invest 100.0

 Profit Loss: 1043.1135284521508e

Expected annual return: 108.8%
Annual volatility: 64.8%
Sharpe Ratio: 1.65
Invest 100.0

 Profit Loss: 1143.673122234091e

Expected annual return: 105.1%
Annual volatility: 62.3%
Sharpe Ratio: 1.65
Invest 100.0

 Profit Loss: 1042.3565613839098e

Expected annual return: 95.2%
Annual volatility: 59.0%
Sharpe Ratio: 1.58
Invest 100.00000000000001

 Profit Loss: 486.1747256465173e

Expected annual return: 94.2%
Annual volatility: 56.4%
Sharpe Ratio: 1.63
Invest 100.0

 Profit Loss: 536.4182719284261e

Expected annual return: 92.8%
Annual volatility: 99.9%
Sharpe Ratio: 0.91
Invest 100.0

 Profit Loss: 679.7893235200798e

Expected annual return: 94.4%
Annual volatility: 91.4%
Sharpe Ratio: 1.01
Invest 100.0

 Profit Loss: 511.78151478090626e

Expe

Expected annual return: 69.4%
Annual volatility: 46.4%
Sharpe Ratio: 1.45
Invest 100.0

 Profit Loss: 384.8632001763958e

Expected annual return: 67.6%
Annual volatility: 52.1%
Sharpe Ratio: 1.26
Invest 100.0

 Profit Loss: 36.903683776240605e

Expected annual return: 70.9%
Annual volatility: 48.1%
Sharpe Ratio: 1.43
Invest 100.0

 Profit Loss: 18.21207423347739e

Expected annual return: 65.3%
Annual volatility: 46.7%
Sharpe Ratio: 1.35
Invest 100.0

 Profit Loss: 4.588993240601762e

Time elapsed: 10.681198434034984


In [16]:
app_wrapper.check_coins(best_portf_2)
print(maxpl2)
optimized_variables["2_coins"]
app_wrapper.get_df_from_dict(best_portf_2)

ETH 9.18997475780779
35810.25740994037


,Date,Coin,Amount,n_coins
0,2019-05-12,ETH,100.0,0.533831
0,2019-06-11,ETH,100.0,0.406865
0,2019-07-11,ETH,100.0,0.372157
0,2019-08-10,ETH,100.0,0.483714
0,2019-09-09,ETH,100.0,0.552030
0,2019-10-09,ETH,100.0,0.517349
0,2019-11-08,ETH,100.0,0.542854
0,2019-12-08,ETH,100.0,0.661093
0,2020-01-07,ETH,100.0,0.696651
0,2020-02-06,ETH,100.0,0.470945


In [17]:
app_wrapper.check_coins(best_portf_10)
print(maxpl10)
optimized_variables["10_coins"]
app_wrapper.get_df_from_dict(best_portf_10)

BNB 102.96766740093607
66728.65812687916


,Date,Coin,Amount,n_coins
0,2019-05-12,BNB,100.0,4.800700
0,2019-06-11,BNB,100.0,3.099559
0,2019-07-11,BNB,100.0,3.373595
0,2019-08-10,BNB,100.0,3.369061
0,2019-09-09,BNB,100.0,4.486842
0,2019-10-09,BNB,100.0,5.615902
0,2019-11-08,BNB,100.0,5.078521
0,2019-12-08,BNB,100.0,6.379779
0,2020-01-07,BNB,100.0,6.662558
0,2020-02-06,BNB,100.0,4.855485


In [18]:
app_wrapper.check_coins(best_portf_20)
print(maxpl20)
optimized_variables["20_coins"]
app_wrapper.get_df_from_dict(best_portf_20)

BNB 102.5078891897537
KLAY 24.489159680003283
SOL 2.019874704198843
66564.7820861753


,Date,Coin,Amount,n_coins
0,2019-05-12,BNB,100.0,4.800700
0,2019-06-11,BNB,100.0,3.099559
0,2019-07-11,BNB,100.0,3.373595
0,2019-08-10,BNB,100.0,3.369061
0,2019-09-09,BNB,100.0,4.486842
0,2019-10-09,BNB,100.0,5.615902
0,2019-11-08,BNB,100.0,5.078521
0,2019-12-08,BNB,100.0,6.379779
0,2020-01-07,BNB,100.0,6.662558
0,2020-02-06,BNB,100.0,4.855485


## Results

In [12]:
print(f'Profit 2c: {round(maxpl2, 2)} $')
print(f'Profit 10c: {round(maxpl10, 2)} $')
print(f'Profit 20c: {round(maxpl20, 2)} $')
# print(f'Profit 100c: {round(maxpl100, 2)} $')

print(f'\nInvestment: {3 * budget * len(n_days_vector)} $')
print(f'Final Profit: {round(maxpl2 + maxpl10 + maxpl20 , 2)} $') # + maxpl100
print(f'Final Profit: {round((maxpl2 + maxpl10 + maxpl20 ) / (3 * budget * len(n_days_vector)) * 100, 2)} %\n') # + maxpl100

end = time()
print(f'Time elapsed: {(end - start) / 60}')

Profit 2c: 35810.26 $
Profit 10c: 66728.66 $
Profit 20c: 66564.78 $

Investment: 7500 $
Final Profit: 169103.7 $
Final Profit: 2254.72 %

Time elapsed: 10.682108012835185


# Summary

## 2 Coins

In [13]:
optimized_variables["2_coins"]

{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic', 'compounding': False}

## 10 Coins

In [14]:
optimized_variables["10_coins"]

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

## 20 Coins

In [15]:
optimized_variables["20_coins"]

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

# Conclusion

## Bear Market 2022

### 2 Coins

{'mu': 'capm',
 's': 'sample',
 'obj_function': 'min_volat',
 'compounding': False}
 
### 10 Coins 

{'mu': 'capm', 's': 'exp', 'obj_function': 'min_volat', 'compounding': False}

### 20 Coins

{'mu': 'capm', 's': 'sample', 'obj_function': 'min_volat', 'compounding': False}

## Bear Market 2018

### 2 Coins:

{'mu': 'capm', 's': 'exp', 'obj_function': 'min_volat', 'compounding': False}

### 10 Coins 

{'mu': 'capm', 's': 'exp', 'obj_function': 'min_volat', 'compounding': False}

### 20 Coins

{'mu': 'mean', 's': 'sample', 'obj_function': 'sharpe', 'compounding': True}

If i had applied these parameters in the bear market of 2022 i would have a profit of ~5%

## Bull Market 2016

### 2 Coins:

{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic', 'compounding': False}


### 10 Coins 

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

### 20 Coins

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}


If i had applied these parameters in the bull market of 2020 i would have a profit of ~2240%

## Bull Market 2020

### 2 Coins:

{'mu': 'capm', 's': 'exp', 'obj_function': 'quadratic', 'compounding': False}


### 10 Coins 

{'mu': 'exp', 's': 'exp', 'obj_function': 'quadratic', 'compounding': True}

### 20 Coins

{'mu': 'exp', 's': 'sample', 'obj_function': 'quadratic', 'compounding': True}
